In [144]:
import pandas as pd
import cvxpy as cp

In [145]:
# CVXPY version: https://kevintcarlberg.net/files/opt_class_icme/7_convexity.pdf

In [190]:
pd.options.display.width = 180

In [191]:
df = pd.read_csv('../data/clean_flyer_data.csv')

In [283]:
# Various sets of columns
cols = ['name', 'cost_per_serving', 'total_fat', 'protein', 'total_carbs']
vcols = cols + ['vegan']
gfcols = cols + ['gluten_free']
kcols = cols + ['kosher']
macro_nutrient_cols = ['total_fat', 'protein', 'total_carbs']

In [284]:
# List for any foods we want to omit
# exclude_list = ['Organic High Protein Tofu']
exclude_list = []

In [285]:
# subset the data by that which isn't empy
tdf = df.loc[~df[cols].isnull().any(axis=1), cols].copy()
vdf = df.loc[(~df[vcols].isnull().any(axis=1)) & (df.vegan==1), vcols].copy()
gfdf = df.loc[(~df[vcols].isnull().any(axis=1)) & (df.gluten_free==1), gfcols].copy()
kdf = df.loc[(~df[vcols].isnull().any(axis=1)) & (df.kosher==1), kcols].copy()

In [286]:
# Convert grams to calories
tdf['protein'] = tdf.protein * 4
tdf['total_carbs'] = tdf.total_carbs * 4
tdf['total_fat'] = tdf.total_fat * 9

vdf['protein'] = vdf.protein * 4
vdf['total_carbs'] = vdf.total_carbs * 4
vdf['total_fat'] = vdf.total_fat * 9

gfdf['protein'] = gfdf.protein * 4
gfdf['total_carbs'] = gfdf.total_carbs * 4
gfdf['total_fat'] = gfdf.total_fat * 9

kdf['protein'] = kdf.protein * 4
kdf['total_carbs'] = kdf.total_carbs * 4
kdf['total_fat'] = kdf.total_fat * 9

In [287]:
# Compute the calories from the macronutrients
tdf['calories'] = tdf[macro_nutrient_cols].sum(axis=1)
vdf['calories'] = vdf[macro_nutrient_cols].sum(axis=1)
gfdf['calories'] = gfdf[macro_nutrient_cols].sum(axis=1)
kdf['calories'] = kdf[macro_nutrient_cols].sum(axis=1)

In [288]:
# tdf = tdf[tdf.interest == 1]
# tdf = tdf[[c for c in cols if c != 'calories'] + ['computed_calories']]
tdf = tdf[~tdf.name.isin(exclude_list)]

In [290]:
# tdf.shape

In [291]:
from tabulate import tabulate

In [292]:
print(tabulate(tdf.head(), tablefmt="pipe", headers="keys"))

|    | name                           |   cost_per_serving |   total_fat |   protein |   total_carbs |   calories |
|---:|:-------------------------------|-------------------:|------------:|----------:|--------------:|-----------:|
|  0 | For the Love of Chocolate Cake |            1.33    |       180   |        16 |           176 |      372   |
|  1 | Pancake Bread                  |            0.49875 |        90   |         8 |           100 |      198   |
|  2 | Pizza Crusts                   |            0.43625 |        22.5 |        20 |           112 |      154.5 |
|  3 | Sprouted Wheat Sourdough       |            0.1995  |         0   |        28 |            56 |       84   |
|  5 | Cold Pressed Green Juice       |            3.99    |         0   |        20 |            80 |      100   |


## The Problem

In [293]:
def create_constraints(A, x, diet):
    """
    Takes a list of diet parameters and 
    returns a list of cvxpy constraint 
    expressions
    """
    min_cals = diet[0]
    max_cals = diet[1]
    fat = diet[2]
    protein = diet[3]
    carbs = diet[4]
    max_servings = diet[5]
    
    return [
        A[:, 0]*x >= min_cals,
        A[:, 0]*x <= max_cals,
        A[:, 1]*x >= min_cals * fat,
        A[:, 1]*x <= max_cals * fat,
        A[:, 2]*x >= min_cals * protein,
        A[:, 2]*x <= max_cals * protein,
        A[:, 3]*x >= min_cals * carbs,
        A[:, 3]*x <= max_cals * carbs,
        x >= 0,
        x <= max_servings
    ]

In [294]:
def create_variables(df, cols, objective_col):
    """
    Takes a dataframe with response information and
    returns a tuple of variables and a problem
    """
    n = df.shape[0]
    c = df[objective_col].values
    A = df[cols].values
    
    return n, c, A

In [295]:
def create_problem(n, c, A, diet):
    """
    Takes a set of variables and a diet and
    returns a vector and a problem
    """
    x = cp.Variable(n, integer=True)
    obj = cp.Minimize(c.T*x)
    constr = create_constraints(A, x, diet)
    problem = cp.Problem(obj, constr)
    
    return x, problem

In [296]:
def create_summary(df, cols):
    """
    Takes a dataframe with serving information
    and returns a dataframe of servings and costs
    and total cost
    """
    for col in cols:
        df.loc[:, col] = df[col] * df.optimal_servings
    df['cost'] = df.cost_per_serving * df.optimal_servings
    total_cost = df.cost.sum()
    return df, total_cost

In [356]:
df['pro_per_dollar'] = (df.protein * df.servings / df.price)
df['fat_per_dollar'] = (df.total_fat * df.servings / df.price)
df['cals_per_dollar'] = (df.calories / df.price)

In [357]:
# df.plot.scatter(x='pro_per_dollar', y='fat_per_dollar')

In [355]:
df.sort_values('pro_per_dollar', ascending=False).head()

,id,url,name,price,servings,serving_size,cost_per_serving,calories,protein,total_fat,...,potassium,vit_a,vit_c,calcium,trans_fat.1,vegan,gluten_free,kosher,pro_per_dollar,fat_per_dollar
258,4930,https://www.traderjoes.com/fearless-flyer/arti...,Creamy Salted Peanut Butter,1.99,14.0,Serving size 2 Tbsp. (32g),0.142143,190.0,7.0,16.0,...,200.0,NaN,NaN,NaN,0.0,0,0,1,49.246231,112.562814
3,4655,https://www.traderjoes.com/fearless-flyer/arti...,Sprouted Wheat Sourdough,3.99,20.0,Serving size 1 slice (24g),0.199500,90.0,7.0,0.0,...,60.0,NaN,NaN,10.0,0.0,0,0,0,35.087719,0.000000
41,4698,https://www.traderjoes.com/fearless-flyer/arti...,Organic High Protein Tofu,2.49,5.0,Serving size 3 oz (84g/about 1/5 pkg),0.498000,130.0,14.0,7.0,...,NaN,NaN,NaN,NaN,0.0,1,0,1,28.112450,14.056225
370,5048,https://www.traderjoes.com/fearless-flyer/arti...,"Pumpkin Pancake Mixes, Gluten-Full or Gluten-Free",1.99,12.0,Serving size 1/3 cup dry mix (50g),0.165833,170.0,4.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0,0,0,24.120603,0.000000
500,5185,https://www.traderjoes.com/fearless-flyer/arti...,Cornish Game Hens,3.99,4.5,Serving size 4oz (112g),0.886667,220.0,19.0,16.0,...,NaN,NaN,NaN,NaN,0.0,0,0,0,21.428571,18.045113


In [349]:
df.sort_values('fat_per_dollar', ascending=False).head()

,id,url,name,price,servings,serving_size,cost_per_serving,calories,protein,total_fat,...,potassium,vit_a,vit_c,calcium,trans_fat.1,vegan,gluten_free,kosher,pro_per_dollar,fat_per_dollar
258,4930,https://www.traderjoes.com/fearless-flyer/arti...,Creamy Salted Peanut Butter,1.99,14.0,Serving size 2 Tbsp. (32g),0.142143,190.0,7.0,16.0,...,200.0,NaN,NaN,NaN,0.0,0,0,1,49.246231,112.562814
107,4769,https://www.traderjoes.com/fearless-flyer/arti...,Organic EVOO from Spain,5.49,33.0,Serving size 1 Tbsp (15mL),0.166364,120.0,0.0,14.0,...,NaN,NaN,NaN,NaN,0.0,0,0,0,0.000000,84.153005
437,5122,https://www.traderjoes.com/fearless-flyer/arti...,Pie Crusts,3.99,16.0,Serving size 1/8 crust (39g),0.249375,190.0,2.0,13.0,...,NaN,NaN,NaN,NaN,0.0,0,0,0,8.020050,52.130326
101,4760,https://www.traderjoes.com/fearless-flyer/arti...,Banana Chips,1.49,8.0,Serving size 1/4 cup (30g/1oz),0.186250,160.0,1.0,9.0,...,NaN,NaN,NaN,NaN,0.0,0,0,0,5.369128,48.322148
237,4908,https://www.traderjoes.com/fearless-flyer/arti...,Salt & Pepper Ridge Cut Potato Chips,2.99,16.0,Serving size 1 oz (28g/about 9 chips),0.186875,150.0,2.0,9.0,...,410.0,NaN,NaN,NaN,0.0,0,1,1,10.702341,48.160535


In [358]:
df.sort_values('cals_per_dollar', ascending=False).head()

,id,url,name,price,servings,serving_size,cost_per_serving,calories,protein,total_fat,...,vit_a,vit_c,calcium,trans_fat.1,vegan,gluten_free,kosher,pro_per_dollar,fat_per_dollar,cals_per_dollar
172,4839,https://www.traderjoes.com/fearless-flyer/arti...,Chocolate Covered Wafer Cookie,0.79,1.0,Serving size 1 bar (55g),0.790,290.0,3.0,17.0,...,NaN,NaN,NaN,0.0,0,0,0,3.797468,21.518987,367.088608
50,4707,https://www.traderjoes.com/fearless-flyer/arti...,Dark Chocolate Sunflower Seed Butter Cups,0.99,1.0,Serving size 2 pieces (40g),0.990,230.0,4.0,18.0,...,NaN,NaN,NaN,0.0,0,1,1,4.040404,18.181818,232.323232
257,4929,https://www.traderjoes.com/fearless-flyer/arti...,Organic Shells with White Cheddar,1.39,2.5,"Serving size 2/3 cup (71g) dry, 1 cup (178g) p...",0.556,260.0,10.0,3.5,...,NaN,NaN,NaN,0.0,0,0,0,17.985612,6.294964,187.050360
504,5189,https://www.traderjoes.com/fearless-flyer/arti...,Egg Nog Whole Milk Greek Yogurt,0.99,1.0,Serving size 1 container (150g/5.3 oz),0.990,180.0,11.0,7.0,...,NaN,NaN,NaN,0.0,0,0,0,11.111111,7.070707,181.818182
318,4993,https://www.traderjoes.com/fearless-flyer/arti...,Apple Caramel & Apple Cinnamon Yogurts,0.99,1.0,Serving size 1 container (150g/5.3oz),0.990,180.0,11.0,6.0,...,NaN,NaN,NaN,0.0,0,0,0,11.111111,6.060606,181.818182


### Balanced, Keto, and Atkins

In [298]:
diets = {
    'Balanced': [2200, 2800, .3, .3, .4, 4],
    'Keto': [2200, 2800, .7, .24, .01, 4], # high fat and protein
    'Atkins': [2200, 2800, .45, .45, .1, 4] # low carb
}

In [299]:
def optimize_diets(df, cols, diet, diet_config):
    n, c, A = create_variables(df, cols[1:], 'cost_per_serving')

    print(diet)
    print()
    # Create the problem
    x, problem = create_problem(n, c, A, diet_config)
    # Solve the problem
    opt_value = problem.solve(verbose=True, solver='GLPK_MI')

    print("Problem status: {}".format(problem.status))
    print()

    # Show the summary
    df['optimal_servings'] = x.value
    # Add info about the dual solution here
    # df['dual'] = problem.constraints[0].dual_value
    # https://www.youtube.com/watch?v=3Q9mMluX3Gw&list=PL3940DD956CDF0622&index=9
    servings, total_cost = create_summary(df[df.optimal_servings > 0].copy(), cols[1:])

    print(tabulate(servings.sort_values('optimal_servings', ascending=False), tablefmt="pipe", headers="keys"))
    print()
    print(servings[cols[1:]].sum())
    print()
    print("% of calories")
    print(servings[cols].sum() / servings.calories.sum() * 100)
    print()
    print("Total cost: ${}".format(round(total_cost, 2)))
    print()
    print()

In [300]:
diet_cols = ['cost_per_serving', 'calories', 'total_fat', 'protein', 'total_carbs']

In [319]:
for diet, diet_params in diets.items():
    optimize_diets(tdf.copy(), diet_cols, diet, diet_params)

Balanced

Problem status: optimal

|     | name                                              |   cost_per_serving |   total_fat |   protein |   total_carbs |   calories |   optimal_servings |     cost |
|----:|:--------------------------------------------------|-------------------:|------------:|----------:|--------------:|-----------:|-------------------:|---------:|
|   3 | Sprouted Wheat Sourdough                          |           0.1995   |           0 |       112 |           224 |        336 |                  4 | 0.798    |
|  41 | Organic High Protein Tofu                         |           0.498    |         252 |       224 |            48 |        524 |                  4 | 1.992    |
| 258 | Creamy Salted Peanut Butter                       |           0.142143 |         432 |        84 |            84 |        600 |                  3 | 0.426429 |
| 370 | Pumpkin Pancake Mixes, Gluten-Full or Gluten-Free |           0.165833 |           0 |        48 |           432 |   

### Vegan Diet

In [304]:
def create_vegan_constraints(A, x, diet):
    """
    Takes a list of diet parameters and 
    returns a list of cvxpy constraint 
    expressions
    """
    min_cals = diet[0]
    max_cals = diet[1]
    max_servings = diet[2]
    
    return [
        A[:, 0]*x >= min_cals,
        A[:, 0]*x <= max_cals,
        x >= 0,
        x <= max_servings
    ]

In [324]:
def optimize_vegan(df, cols, constr_config):
    n, c, A = create_variables(df, cols[1:], 'cost_per_serving')
    
    print('Minimize cost for vegan foods')
    print()
    
    # Variable, constraints, and problem
    x = cp.Variable(n, integer=True)
    obj = cp.Minimize(c.T*x)
    constr = create_vegan_constraints(A, x, constr_config)
    problem = cp.Problem(obj, constr)

    # Solve the problem and show the output
    opt_value = problem.solve(verbose=True)
    df['optimal_servings'] = x.value
    servings, total_cost = create_summary(df[df.optimal_servings > 0].copy(), cols[1:])

    
    print(tabulate(servings.sort_values('optimal_servings', ascending=False), tablefmt="pipe", headers="keys"))
    print()
    print(servings.sum())
    print()
    print(servings[cols[1:]].sum())
    print()
    print("% of calories")
    print(servings[cols].sum() / servings.calories.sum() * 100)
    print()
    print("Total cost: ${}".format(round(total_cost, 2)))
    print()

In [325]:
vegan_cols = ['cost_per_serving', 'calories', 'total_fat', 'protein', 'total_carbs']

In [326]:
vegan_constr_config = [2200, 2800, 4]

In [327]:
optimize_vegan(vdf.copy(), vegan_cols, vegan_constr_config)

Minimize cost for vegan foods

|     | name                            |   cost_per_serving |   total_fat |   protein |   total_carbs |   vegan |   calories |   optimal_servings |     cost |
|----:|:--------------------------------|-------------------:|------------:|----------:|--------------:|--------:|-----------:|-------------------:|---------:|
|  41 | Organic High Protein Tofu       |           0.498    |       252   |       224 |            48 |       1 |      524   |                  4 | 1.992    |
|  83 | Japanese Style Fried Rice       |           0.854286 |       180   |       112 |           672 |       1 |      964   |                  4 | 3.41714  |
| 344 | Vegan Banana Bread with Walnuts |           0.49875  |       243   |        24 |           360 |       1 |      627   |                  3 | 1.49625  |
| 273 | Soft-Baked Snickerdoodles       |           0.498333 |        40.5 |         4 |            72 |       1 |      116.5 |                  1 | 0.498333 |

name    

### Gluten Free

In [328]:
def optimize_gluten_free(df, cols, constr_config):
    n, c, A = create_variables(df, cols[1:], 'cost_per_serving')
    
    print('Minimize cost for vegan foods')
    print()
    
    # Variable, constraints, and problem
    x = cp.Variable(n, integer=True)
    obj = cp.Minimize(c.T*x)
    constr = create_constraints(A, x, constr_config)
    problem = cp.Problem(obj, constr)

    # Solve the problem and show the output
    opt_value = problem.solve(verbose=True)
    df['optimal_servings'] = x.value
    servings, total_cost = create_summary(df[df.optimal_servings > 0].copy(), cols[1:])

    
    print(tabulate(servings.sort_values('optimal_servings', ascending=False), tablefmt="pipe", headers="keys"))
    print()
    print(servings.sum())
    print()
    print(servings[cols[1:]].sum())
    print()
    print("% of calories")
    print(servings[cols].sum() / servings.calories.sum() * 100)
    print()    
    print("Total cost: ${}".format(round(total_cost, 2)))
    print()

In [329]:
gfdf.shape

(21, 7)

In [330]:
gf_constr_config = [2200, 2800, .3, .2, .5, 4]

In [331]:
optimize_gluten_free(gfdf.copy(), vegan_cols, gf_constr_config)

Minimize cost for vegan foods

|     | name                                    |   cost_per_serving |   total_fat |   protein |   total_carbs |   gluten_free |   calories |   optimal_servings |     cost |
|----:|:----------------------------------------|-------------------:|------------:|----------:|--------------:|--------------:|-----------:|-------------------:|---------:|
|   2 | Pizza Crusts                            |           0.43625  |          90 |        80 |           448 |             1 |        618 |                  4 | 1.745    |
|  35 | Organic Creamy Tomato Soup              |           0.6725   |          72 |        80 |           256 |             1 |        408 |                  4 | 2.69     |
| 143 | Neapolitan Puffs Cereal                 |           0.527143 |          54 |        96 |           432 |             1 |        582 |                  4 | 2.10857  |
| 226 | Organic Grass-Fed Uncured Beef Hot Dogs |           1.198    |         360 |       144 |   

### Minimize Sodium with Max Cost of $N / day

In [143]:
def create_sodium_constraints(A, x, diet):
    """
    Takes a list of diet parameters and 
    returns a list of cvxpy constraint 
    expressions
    """
    min_cals = diet[0]
    max_cost = diet[1]
    fat = diet[2]
    protein = diet[3]
    carbs = diet[4]
    sugars = diet[5]
    max_servings = diet[6]
    
    return [
        A[:, 0]*x >= min_cost,
        A[:, 1]*x <= max_cost,
        A[:, 2]*x >= min_cals * fat,
        A[:, 3]*x >= min_cals * protein,
        A[:, 4]*x >= min_cals * carbs,
        A[:, 5]*x <= sugars,
        x >= 0,
        x <= max_servings
    ]

In [144]:
def optimize_sodium(df, cols, constr_config):
    n, c, A = create_variables(df, cols[1:], 'sodium')
    
    print('Minimize sodium with Cost Constraint')
    print()
    print(constr_config)
    
    # Variable, constraints, and problem
    x = cp.Variable(n, integer=True)
    obj = cp.Minimize(c.T*x)
    constr = create_sodium_constraints(A, x, constr_config)
    problem = cp.Problem(obj, constr)

    # Solve the problem and show the output
    opt_value = problem.solve(verbose=True)
    df['servings'] = x.value
#     df['dual'] = problem.constraints[0].dual_value
    servings, total_cost = create_summary(df[df.servings > 0].copy(), cols)
    
#     for constr in problem.constraints:
#         print(constr.dual_value)
    
    print(servings)
    print()
    print(servings[cols[1:]].sum())
    print()
    print("% of calories")
    print(servings[cols].sum() / servings.computed_calories.sum() * 100)
    print()
    print("Total cost: ${}".format(round(total_cost, 2)))
    print()

In [145]:
min_sodium_cols = ['sodium', 'computed_calories', 'cost_per_serving', 'total_fat', 'protein', 'total_carbs', 'sugars']

In [146]:
sodium_constr_config = [2400, 8, .3, .3, .4, 200, 7]

In [147]:
configs = [sodium_constr_config + [c] for c in range(1, 10)]

In [65]:
optimize_sodium(tdf.copy(), min_sodium_cols, sodium_constr_config)

Minimize sodium with Cost Constraint

[2400, 8, 0.3, 0.3, 0.4, 200, 7]
                                 name  cost_per_serving  total_fat  protein  total_carbs  sodium  sugars  fiber  computed_calories  servings       cost
30           Sprouted Wheat Sourdough          0.798000        0.0    112.0        224.0   170.0     4.0    2.0              336.0       4.0   3.192000
68          Organic High Protein Tofu          3.486000      441.0    392.0         84.0    15.0     0.0    0.0              917.0       7.0  24.402000
81   Organic Blue Corn Tortilla Chips          0.996667      324.0     48.0        272.0    40.0     0.0    2.0              644.0       4.0   3.986667
128                      Banana Chips          0.931250      405.0     20.0        380.0     0.0    15.0    2.0              805.0       5.0   4.656250
528                 Cornish Game Hens          1.773333      288.0    152.0          0.0    70.0     0.0    0.0              440.0       2.0   3.546667

computed_calorie

In [82]:
# for config in configs:
#     optimize_sodium(tdf.copy(), min_sodium_cols, config)